## Building a Period Prediction Linear Regression Model

This notebook demonstrates the process of building a linear regression model to predict menstrual cycle periods.

**Dataset:** 
The dataset used in this analysis was obtained from Kaggle
#### Purpose
With this, girls can predict their next period, stand out without what if? and now the changes that goes on in their bodies 


In [29]:
#importing EDA libraries 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#import Ml libraries 

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split


In [3]:
#importing dataset

data = pd.read_csv("menstrual_tracking_data.csv")

## EXPLORING THE DATASET

In [25]:
#displaying the first 5 rows
data.head(5)

,user_id,log_date,cycle_length,period_duration,symptoms
0,1,2023-08-18,27,4,"bloating, fatigue, acne"
1,2,2023-07-08,24,3,"acne, bloating"
2,3,2023-03-01,33,7,"cramps, bloating"
3,4,2023-09-05,35,6,"bloating, cramps, fatigue"
4,5,2023-06-02,31,4,fatigue


In [23]:
#displaying the column names
data.columns

Index(['user_id', 'log_date', 'cycle_length', 'period_duration', 'symptoms'], dtype='object')

In [ ]:
#information about the dataset
data.info()
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          500 non-null    int64 
 1   log_date         500 non-null    object
 2   cycle_length     500 non-null    int64 
 3   period_duration  500 non-null    int64 
 4   symptoms         500 non-null    object
dtypes: int64(3), object(2)
memory usage: 19.7+ KB


(500, 5)

In [22]:
#describing numerical values 
data.describe()

,user_id,cycle_length,period_duration
count,500.000000,500.000000,500.000000
mean,250.500000,27.846000,5.032000
std,144.481833,4.367199,1.461248
min,1.000000,21.000000,3.000000
25%,125.750000,24.000000,4.000000
50%,250.500000,28.000000,5.000000
75%,375.250000,32.000000,6.000000
max,500.000000,35.000000,7.000000


In [21]:
#describing non-numerical values 
data.describe(include='O')

,log_date,symptoms
count,500,500
unique,359,128
top,2023-09-08,acne
freq,5,32


In [20]:
#checking for null values 
data.isnull()
data.isnull().sum()
##

user_id            0
log_date           0
cycle_length       0
period_duration    0
symptoms           0
dtype: int64

In [28]:
data[["cycle_length"]].max()

cycle_length    35
dtype: int64